In [ ]:
%pip uninstall matplotlib 
%pip install matplotlib --force-reinstall --user

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import pandas_profiling as pp
import sweetviz as sv

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

ImportError: cannot import name 'animation' from partially initialized module 'matplotlib' (most likely due to a circular import) (d:\Users\Phillip\anaconda3\lib\site-packages\matplotlib\__init__.py)

In [ ]:
measure1 = pd.read_csv('./data/measurements.csv')
measure1.columns = [i.lower().replace(' ', '_') for i in measure1.columns]
measure1

In [ ]:
measure2 = pd.read_excel('./data/measurements2.xlsx')
measure2.columns = [i.lower().replace(' ', '_') for i in measure2.columns]
measure2

In [ ]:
# profile1 = pp.ProfileReport(measure1, minimal=True)
# profile1.to_file("measurements1.html")

In [ ]:
# profile2 = pp.ProfileReport(measure2, minimal=True)
# profile2.to_file("measurements2.html")

In [ ]:
## comparing both datasets showed that they are the same, just in a different format, therefore I will use measure2, as the datatypes are more appropriate already.

In [ ]:
final = measure2

In [ ]:
final

In [ ]:
final.isna().sum()

In [ ]:
final[final['temp_inside'].isna()]

In [ ]:
final.specials.value_counts()

In [ ]:
final.loc[final['specials'] == 'AC rain']

In [ ]:
final.loc[final['specials'] == 'snow']

In [ ]:
### checking the specials column like this shows that there is only duplicate information, since we got information on ac/rain/sun in other columns, snow is treated as rain. Therefore I drop the specials column.

In [ ]:
final[final['specials'].isna()]

In [ ]:
final.drop(['specials'],axis=1,inplace=True)

In [ ]:
## Missing values in "refill_liters" occur when there was no refill, therefore replaced with 0

In [ ]:
final['refill_liters'].fillna('0',inplace=True)

In [ ]:
final

In [ ]:
final['refill_gas'].fillna('None',inplace=True)

In [ ]:
final

In [ ]:
final['temp_inside'].fillna(22 ,inplace=True) # filling the 12 missing cells in temp inside with the median value of 22

In [ ]:
# profile = pp.ProfileReport(final, minimal=True)
# profile.to_file("final.html")

In [ ]:
final.loc[final['refill_gas'] != 'None']

In [ ]:
final.groupby('gas_type', as_index=False)['consume'].mean()

In [ ]:
final.groupby('gas_type', as_index=False)['distance'].mean()

In [ ]:
final.groupby('gas_type', as_index=False)['speed'].mean()

In [ ]:
final["trip_consumption"] = final["distance"] * final["consume"] / 100

In [ ]:
final

In [ ]:
final.columns

In [ ]:
final = final[['distance', 'consume', 'speed', 'trip_consumption','temp_inside', 'temp_outside',
       'gas_type', 'ac', 'rain', 'sun', 'refill_liters', 'refill_gas',
       ]]

In [ ]:
final['trip_consumption'] = final['trip_consumption'].round(2)

In [ ]:
final

In [ ]:
final.loc[final['refill_gas'] != 'None']

In [ ]:
final["trip_number"] = np.nan

In [ ]:
final.loc[0:43,'trip_number'] = 1
final.loc[44:81,'trip_number'] = 2
final.loc[82:105,'trip_number'] = 3
final.loc[106:138,'trip_number'] = 4
final.loc[139:170,'trip_number'] = 5
final.loc[171:190,'trip_number'] = 6
final.loc[191:191,'trip_number'] = 7
final.loc[192:233,'trip_number'] = 8
final.loc[234:273,'trip_number'] = 9
final.loc[274:312,'trip_number'] = 10
final.loc[313:324,'trip_number'] = 11
final.loc[325:348,'trip_number'] = 12
final.loc[349:,'trip_number'] = 13

In [ ]:
final

In [ ]:
final.groupby('trip_number', as_index=False)['distance'].mean()

In [ ]:
final.groupby('trip_number', as_index=False)['consume'].mean()

In [ ]:
final.groupby('trip_number', as_index=False)['speed'].mean()

In [ ]:
final.to_csv('./final.csv', index=False)

In [ ]:
final

In [ ]:
final['km_absolute'] = final['distance'].cumsum()

In [ ]:
final

In [ ]:
sns.lmplot(data=final,x='km_absolute',y='distance', hue='gas_type', height=8, aspect=1.5,)
plt.xlabel('Km_absolute [Km]')
plt.ylabel('distance [Km]')
plt.savefig("Graph.png")

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [ ]:
X = final.drop(['consume','trip_consumption','refill_liters','refill_gas','trip_number'], axis = 1)
y = final['consume'] 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
numerical_train = X_train.select_dtypes(np.number)
numerical_test = X_test.select_dtypes(np.number)
categorical_train= X_train.select_dtypes(object)
categorical_test= X_test.select_dtypes(object)

In [ ]:
transformer = MinMaxScaler().fit(numerical_train)
numerical_train_standardized = pd.DataFrame(transformer.transform(numerical_train),index=numerical_train.index, columns = numerical_train.columns)
numerical_test_standardized = pd.DataFrame(transformer.transform(numerical_test),index=numerical_test.index, columns = numerical_test.columns)

In [ ]:
encoder = OneHotEncoder(drop='first',handle_unknown='error').fit(categorical_train)
categorical_train_encoded = encoder.transform(categorical_train).toarray()
categorical_test_encoded = encoder.transform(categorical_test).toarray()
cols = encoder.get_feature_names(input_features=categorical_train.columns)
categorical_train_encoded = pd.DataFrame(categorical_train_encoded, index=categorical_train.index, columns=[i.lower() for i in cols])
categorical_test_encoded = pd.DataFrame(categorical_test_encoded, index=categorical_test.index, columns=[i.lower() for i in cols])


In [ ]:
X_train = pd.concat((numerical_train_standardized,categorical_train_encoded),axis=1)
X_test = pd.concat((numerical_test_standardized,categorical_test_encoded),axis=1)

In [ ]:
from xgboost import XGBRegressor
from xgboost import XGBClassifier

In [ ]:
model = XGBRegressor(
    max_depth=6,
    n_estimators=100,
    min_child_weight=0.5, 
    colsample_bytree=0.5, 
    subsample=0.5, 
    eta=0.1,
    seed=42)

In [ ]:
model.fit(
    X_train, 
    y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, y_train), (X_test, y_test)], 
    verbose=True, 
    early_stopping_rounds = 40)

In [ ]:
xgb.plot_importance(model)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,10))
xgb.plot_importance(model, max_num_features=5, ax=ax)

In [ ]:
model.get_booster().get_fscore() 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV


max_depth_choices= [1,3,6,10,None]
eval_metric = ['rmse']
min_child_weight = [0.1,0.3,0.5,0.7,0.9,1]
colsample_bytree = [0.1,0.3,0.5,0.7,0.9,1]
subsample = [0.1,0.3,0.5,0.7,0.9,1]
eta= [0.1,0.2,0.3]


random_grid = {'max_depth': max_depth_choices,
                'eval_metric': eval_metric,
               'min_child_weight': min_child_weight,
               'colsample_bytree': colsample_bytree,
               'subsample': subsample,
                'eta': eta }

In [ ]:
model = XGBRegressor(
    n_estimators=100,
    seed=42)

In [ ]:
random_search = RandomizedSearchCV(estimator = model, param_distributions = random_grid, n_iter=25, cv = 2, n_jobs = 10)

In [ ]:
random_search.fit(X_train,y_train,verbose=1)

In [ ]:
random_search.best_params_